## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown


In [2]:
load_dotenv(override=True)


True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))


In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems. LangChain is a prominent open-source framework that enables developers to build applications powered by large language models (LLMs). Its modular design supports integration with various LLMs, prompt templates, agent modules for task automation, memory systems for context retention, and dynamic retrieval components for real-time data access. This versatility makes LangChain suitable for applications such as conversational agents, document analysis, and code generation. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

Another notable framework is LangGraph, an extension of LangChain that focuses on creating stateful, graph-based AI agent systems. It employs a graph-based approach to define workflows of prompts and agent interactions, offering explicit control over information flow and enabling branching and debugging of complex agent behaviors. LangGraph is particularly effective for applications requiring long-term context maintenance and potential human intervention, such as complex customer service flows and guided decision-making systems. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))

AutoGen, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It is tightly integrated with Microsoft's Semantic Kernel and Azure services, making it suitable for enterprise-level applications needing event-driven workflows, such as customer service automation. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

CrewAI adopts a role-based agent collaboration approach, facilitating the creation of specialized agents that collaborate on complex projects akin to a team environment. It offers dynamic task planning, real-time performance monitoring, and orchestration of a variety of agents as distinct workers, making it ideal for complex projects requiring teamwork among agents, such as software development or project management. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

Eliza is an open-source, Web3-friendly AI agent operating system that seamlessly integrates with Web3 applications, enabling the deployment of decentralized applications. It emphasizes user control, with all aspects being regular TypeScript programs, and integrates with Web3 technologies like blockchain data and smart contracts. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

Agent Lightning is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent. It achieves complete decoupling between agent execution and training, allowing seamless integration with existing agents developed via diverse methods, with almost zero code modifications. This framework is suitable for real-world agent training and deployment, demonstrating stable, continuous improvements across various tasks. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

AutoAgent is a fully-automated, zero-code framework for LLM agents, enabling users to create and deploy LLM agents through natural language alone. It comprises components like Agentic System Utilities, an LLM-powered Actionable Engine, a Self-Managing File System, and a Self-Play Agent Customization module, facilitating efficient and dynamic creation and modification of tools, agents, and workflows without coding requirements. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each contributing to the advancement of intelligent, autonomous systems across various domains. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(
        description="Your reasoning for why this search is important to the query."
    )

    query: str = Field(
        description="The search term to use for the web search."
    )


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="A list of web searches to perform to best answer the query."
    )


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)


In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)


searches=[WebSearchItem(reason='To find updated information on popular AI agent frameworks released or gaining traction in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore articles, blogs, and research papers discussing advancements and trends in AI agent technologies in 2025.', query='AI agent technology trends 2025'), WebSearchItem(reason='To identify key features and comparisons of the leading AI agent frameworks available in 2025.', query='best AI agent frameworks comparison 2025')]


In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114ddf7e0>, strict_json_schema=True, is_enabled=True)

In [9]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(
        description="A short 2-3 sentence summary of the findings."
    )

    markdown_report: str = Field(
        description="The final report"
    )

    follow_up_questions: list[str] = Field(
        description="Suggested topics to research further"
    )


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(
        planner_agent, f"Query: {query}"
    )
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [15]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    print(report)
    # await send_email(report)
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
short_summary='In 2025, several innovative AI agent frameworks have transformed the landscape of intelligent systems. Key developments include Agent Lightning for modular reinforcement learning, AutoAgent as a zero-code solution, and Eliza for Web3 integration, all reflecting a shift towards more autonomous, context-aware, and capable agents.' markdown_report="# Report on Latest AI Agent Frameworks in 2025\n\n## Table of Contents\n1. Introduction  \n2. Overview of AI Agent Frameworks  \n   - 2.1 Agent Lightning  \n   - 2.2 AutoAgent  \n   - 2.3 Eliza  \n   - 2.4 AgentMesh  \n3. Key Trends in AI Agent Development  \n   - 3.1 Transition to Agentic AI  \n   - 3.2 Emphasis on Multimodal Capabilities  \n   - 3.3 The Importance of Emotional Intelligence  \n   - 3.4 Move towards Decentralized AI  \n   - 3.5 Enhancements in Enterprise Integration  \

In [17]:
display(Markdown(report.markdown_report))

# Report on Latest AI Agent Frameworks in 2025

## Table of Contents
1. Introduction  
2. Overview of AI Agent Frameworks  
   - 2.1 Agent Lightning  
   - 2.2 AutoAgent  
   - 2.3 Eliza  
   - 2.4 AgentMesh  
3. Key Trends in AI Agent Development  
   - 3.1 Transition to Agentic AI  
   - 3.2 Emphasis on Multimodal Capabilities  
   - 3.3 The Importance of Emotional Intelligence  
   - 3.4 Move towards Decentralized AI  
   - 3.5 Enhancements in Enterprise Integration  
4. Challenges Facing AI Agent Frameworks  
5. Conclusion  
6. Follow Up Questions  

---

## 1. Introduction  
As we navigate through 2025, the world of artificial intelligence (AI) is experiencing rapid transformations, particularly in the domain of AI agents. These agents, designed to operate autonomously and enhance human capabilities, are being embedded in various sectors, leading to more advanced systems characterized by unique frameworks tailored to specific tasks and needs. This report delves into the latest AI agent frameworks, highlighting their features, key trends, and the challenges that lie ahead.

## 2. Overview of AI Agent Frameworks  
Several frameworks have emerged that showcase distinct capabilities for developing and deploying intelligent agents. Below, we outline some of the leading frameworks in 2025.

### 2.1 Agent Lightning  
Agent Lightning distinguishes itself by enabling the reinforcement learning-based training of large language models (LLMs). This flexible framework decouples agent execution from training, allowing seamless integration with existing agents developed via platforms such as LangChain and OpenAI Agents SDK. It formulates agent execution as a Markov decision process and introduces **LightningRL**, a hierarchical reinforcement learning algorithm that allows for complex interaction logic, including dynamic workflows and multi-agent scenarios. This versatility positions it as a powerful tool for developers aiming to create sophisticated AI agents.

### 2.2 AutoAgent  
Marketing itself as a fully automated, zero-code framework for LLM agents, AutoAgent empowers users to create and deploy agents using natural language instructions. It functions as an autonomous Agent Operating System and includes components like the Agentic System Utilities, an LLM-powered Actionable Engine, and a Self-Managing File System. The system's architecture promotes efficiency, enabling users to design tools, agents, and workflows without the need for coding or manual intervention. This democratizes AI agent development, making it accessible to a broader audience.

### 2.3 Eliza  
Eliza stands out as an open-source, Web3-friendly AI agent operating system. Its design allows smooth integration with decentralized applications, promoting the deployment of smart contracts and other blockchain interactions. Every element of Eliza's runtime operates as a regular TypeScript program under complete user control, enhancing transparency and enabling unique functionalities. This framework caters to developers interested in harnessing the growing potential of Web3 technologies.

### 2.4 AgentMesh  
AgentMesh employs a cooperative approach with Python-based agents to streamline software development tasks. By utilizing various specialized agents—including a Planner, Coder, Debugger, and Reviewer—this framework helps transform high-level requirements into executable code. The collaboration among agents minimizes the limitations associated with single-agent systems and leverages the advantages of LLMs to enhance productivity in software development.

## 3. Key Trends in AI Agent Development  
The progression of AI agents is marked by key trends that redefine the capabilities of these systems in 2025.

### 3.1 Transition to Agentic AI  
AI agents are moving from reactive to proactive models. This shift, often referred to as Agentic AI, enables agents to understand goals, orchestrate tasks, and adjust strategies in real-time. The collaboration between humans and machines is deepening, fostering a more interactive and intuitive workflow.

### 3.2 Emphasis on Multimodal Capabilities  
AI agents are increasingly capable of processing and generating content across multiple modalities—text, images, audio, video, and structured data. This multimodal approach enhances their ability to engage more naturally with users and perform complex tasks that require integrating different types of information.

### 3.3 The Importance of Emotional Intelligence  
There is a growing emphasis on the emotional intelligence of AI agents. This development allows for more empathetic interactions with users, improving customer service and user experience. Agents that can understand human emotions and context better can foster deeper, more meaningful interactions.

### 3.4 Move towards Decentralized AI  
Decentralization is a vital trend that reduces reliance on cloud processing, allowing AI systems to operate on local devices. This shift not only improves privacy and security but also reduces latency, which leads to faster responses and better user experiences.

### 3.5 Enhancements in Enterprise Integration  
Organizations are increasingly integrating AI agents into complex enterprise solutions, automating workflows, and facilitating improved decision-making. For instance, solutions like SAP's Joule utilize collaborative AI agents to streamline business processes and enhance efficiency.

## 4. Challenges Facing AI Agent Frameworks  
Despite the rapid advancements, several challenges persist that organizations must address to fully harness the power of AI agents. Key challenges include:
- **Data Quality:** The performance of AI agents heavily relies on the quality of data they are trained on, necessitating ongoing efforts to ensure data integrity and relevance.
- **Computing Resources:** The computational requirements for training and maintaining these agents can be substantial, posing issues for organizations lacking sufficient infrastructure.
- **Trust and Security:** Ensuring the safety of AI operations is paramount. Vulnerabilities, including prompt injection and tool permissions issues as identified in the Model Context Protocol (MCP), must be diligently mitigated.
- **Ethical Considerations:** Organizations are challenged to implement AI agents responsibly, maintaining transparency and adhering to ethical standards while leveraging these technologies for competitive advantage.

## 5. Conclusion  
The advancements in AI agent frameworks by 2025 signify a transformative shift towards more autonomous and capable systems. With frameworks like Agent Lightning, AutoAgent, and Eliza leading the charge, organizations are poised to harness the full potential of AI agents to improve efficiency and enhance the collaborative workflow. However, to fully realize this potential, organizations must strategically address the associated challenges, including data quality, security, and ethical considerations.

## 6. Follow Up Questions  
- What are the specific use cases for each of the highlighted frameworks in different industries?  
- How are organizations addressing the ethical considerations surrounding AI agent deployment?  
- What future developments can be expected in AI agent capabilities beyond 2025?  
- How will data privacy regulations impact the development and deployment of AI agents?  
- What role will emerging technologies like quantum computing play in the future of AI agents?  

---

This report demonstrates the rapid evolution of AI agents and the frameworks that support them. As technology continues to advance, it will be essential for organizations to adapt and strategically implement these innovations to maximize their benefits.

In [22]:
for question in report.follow_up_questions:
    print(question, '\n')


What are the specific use cases for each of the highlighted frameworks in different industries? 

How are organizations addressing the ethical considerations surrounding AI agent deployment? 

What future developments can be expected in AI agent capabilities beyond 2025? 

How will data privacy regulations impact the development and deployment of AI agents? 

What role will emerging technologies like quantum computing play in the future of AI agents? 



### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>